# Combine weather data set

## Objectives

- Write the merged table to Databricks catalog if there are any difference between the previous and new versions

## Load multiple raw weather data sets

In [0]:
from weather_data_sets import *

weather_data_sets = weather_data_sets_2024

In [0]:
weather_data_dict = {}

for station, table_name in weather_data_sets.items():
    weather_data_dict[station] = spark.table(table_name)

weather_data_dict

## Combine multiple raw weather data sets

In [0]:
from merge_weather_station_data_sets import mergeWeatherData

merged_weather_data_set_sdf = mergeWeatherData(weather_data_dict)

merged_weather_data_set_sdf.printSchema()
merged_weather_data_set_sdf.show()
merged_weather_data_set_sdf.count()

In [0]:
merged_weather_data_set_sdf.createOrReplaceTempView("merged_weather_data_set")
  
spark.sql(
"""CREATE TABLE IF NOT EXISTS datalake_lab.lab_project_fruitdropzgs.merged_weather_data_set_2024 
AS SELECT * FROM merged_weather_data_set"""
)

In [0]:
target_table_name = "datalake_lab.lab_project_fruitdropzgs.merged_weather_data_set_2024"
target_table_sdf = spark.table(target_table_name)
target_table_sdf.printSchema()
target_table_sdf.show()

## Write table to Databricks catalog

In [0]:
from merged_weather_data_expected_schema import merged_weather_data_expected_schema

merged_weather_data_expected_schema

In [0]:
if target_table_sdf.exceptAll(merged_weather_data_set_sdf).count() == 0 and merged_weather_data_set_sdf.exceptAll(target_table_sdf).count() == 0 and merged_weather_data_expected_schema == target_table_sdf.schema:
    print("The dataframes are equal. The merged weather data is up-to-date.")
else:
    merged_weather_data_set_sdf.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(target_table_name)
    print("The datarames are not equal. Overwrite the new version of table")

## Retrieve table history 

In [0]:
spark.sql(f"""DESCRIBE HISTORY {target_table_name}""").show()